<h1>Brief IMDB<h1>

<h2>Auteurs : Kévin Heugas & Thierry Massart<h2>

<h2>Date : 04/2024 (S15)<h2>

[Lien vers Simplonline : Prédiction de note IMDB de film](https://simplonline.co/briefs/53911d01-6ba4-4ad6-90e0-daf34da67ec2)

<h3>1. Importation, exploration/analyse, nettoyage et enregistrement du DataSet<h3>

<h4>1.1. Importation<h4>

In [ ]:
# Importation des librairies python
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

In [ ]:
# Chargement des données IMDB
df = pd.read_csv("5000_movies_bis.csv")

<h4>1.2. Exploration/analyse<h4>

In [ ]:
# Affichage des premières lignes du DataFrame pour comprendre sa structure
pd.set_option('display.max_columns', None)
df.head()

In [ ]:
# Dimensions du DataFrame
df.shape

In [ ]:
# Inspection plus en détail pour certaines colonnes : 'language' et 'content_rating'
df['content_rating'].unique()

In [ ]:
df['language'].unique()

In [ ]:
df['language'].value_counts()

In [ ]:
# Vérification des informations sur les colonnes et les valeurs manquantes
df.info()

In [ ]:
# Vérification des statistiques descriptives pour les colonnes numériques
df.describe()

In [ ]:
# Identification des valeurs manquantes
df.isnull().sum()

In [ ]:
# Exclusion des colonnes non numériques
numeric_df = df.select_dtypes(include=['float64', 'int64'])

# Calcul de la matrice de corrélation
correlation_matrix = numeric_df.corr()

# Affichage du heatmap de la matrice de corrélation
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Matrice de Corrélation')
plt.show()

# Traçage des boxplots pour chaque variable numérique
plt.figure(figsize=(12, 8))
sns.boxplot(data=numeric_df, orient="h")
plt.title('Boîtes à moustaches pour les variables numériques')
plt.show()

<h4>1.3. Nettoyage<h4>

In [ ]:
# Copie du DataFrame d'origine dans un nouveau DataFrame 'df_cleaned'
df_cleaned = df.copy()

In [ ]:
# Suppression des doublons (perte de 45 lignes)
df_cleaned = df_cleaned.drop_duplicates()
df_cleaned.shape

In [ ]:
# Remplacement des valeurs manquantes dans les colonnes numériques par la médiane
numeric_cols = df_cleaned.select_dtypes(include=['float64', 'int64']).columns
df_cleaned[numeric_cols] = df_cleaned[numeric_cols].fillna(df_cleaned[numeric_cols].median())

# Remplacement des valeurs manquantes dans les colonnes de type objet par 'N/C'
object_cols = df_cleaned.select_dtypes(include=['object']).columns
df_cleaned[object_cols] = df_cleaned[object_cols].fillna('N/C')

# Suppression des colonnes non exhaustives (suivant leurs corrélations et choix d'équipe)
df_cleaned = df_cleaned.drop(['color', 'duration', 'director_fb_likes', 'actor_3_fb_likes', 'actor_3_name', 'facenumber_in_poster', 'plot_keywords', 'movie_imdb_link', 'language', 'country', 'content_rating', 'budget', 'title_year', 'aspect_ratio'], axis=1, errors='ignore')

# Re-identification des valeurs manquantes pour validation
df_cleaned.isnull().sum()

In [ ]:
# Test get_dummies VS factorize => factorisation choisie

# genres_dummies = df_cleaned['genres'].str.get_dummies(sep='|')
# df_cleaned = pd.concat([df_cleaned, genres_dummies], axis=1)

# plot_keywords_dummies = df_cleaned['plot_keywords'].str.get_dummies(sep='|')
# df_cleaned = pd.concat([df_cleaned, plot_keywords_dummies], axis=1)

# df_cleaned = df_cleaned.drop(['plot_keywords', 'genres'], axis=1, errors='ignore')

# director_name_dummies = df_cleaned['director_name'].str.get_dummies(sep='|')
# df_cleaned = pd.concat([df_cleaned, director_name_dummies], axis=1)

# actor_2_name_dummies = df_cleaned['actor_2_name'].str.get_dummies(sep='|')
# df_cleaned = pd.concat([df_cleaned, actor_2_name_dummies], axis=1)

# actor_1_name_dummies = df_cleaned['actor_1_name'].str.get_dummies(sep='|')
# df_cleaned = pd.concat([df_cleaned, actor_1_name_dummies], axis=1)

# country_dummies = df_cleaned['country'].str.get_dummies(sep='|')
# df_cleaned = pd.concat([df_cleaned, country_dummies], axis=1)

# language_dummies = df_cleaned['language'].astype(str).str.get_dummies(sep='|')
# df_cleaned = pd.concat([df_cleaned, language_dummies], axis=1)

# df_cleaned = df_cleaned.drop(['director_name', 'actor_2_name', 'actor_1_name', 'country'], axis=1, errors='ignore')
# df_cleaned

In [ ]:
# Factorisation des colonnes
df_cleaned['director_name_factorized'] = df_cleaned['director_name'].factorize()[0]
df_cleaned['actor_2_name_factorized'] = df_cleaned['actor_2_name'].factorize()[0]
df_cleaned['genres_factorized'] = df_cleaned['genres'].factorize()[0]
df_cleaned['actor_1_name_factorized'] = df_cleaned['actor_1_name'].factorize()[0]

# Suppression des colonnes d'origine
df_cleaned = df_cleaned.drop(['director_name', 'actor_2_name', 'genres', 'actor_1_name'], axis=1, errors='ignore')

# Affichage
df_cleaned

# Au final : suppression de 14 colonnes et 45 lignes

<h4>1.4. Enregistrement<h4>

In [ ]:
# Enregistrer du dataset nettoyé dans un nouveau fichier csv
df_cleaned.to_csv("5000_movies_cleaned.csv", index=False)

<h3>2. Modélisation ML<h3>